In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('D:\Project\crop_pred\crop_production.csv')
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [4]:
df.dropna(inplace=True)

In [5]:
ls = LabelEncoder()
df['State_Name'] = ls.fit_transform(df['State_Name'])
df['District_Name'] = ls.fit_transform(df['District_Name'])
df['Season'] = ls.fit_transform(df['Season'])
df['Crop'] = ls.fit_transform(df['Crop'])

In [6]:
df.drop('Crop_Year', inplace=True, axis =1)

In [7]:
df['Production'] = df['Production']/df['Area']

In [8]:
upper_lim = df.Production.quantile(0.75)
lower_lim = df.Production.quantile(0.25)
df = df.loc[(df['Production'] > lower_lim) & (df['Production'] < upper_lim)]

In [9]:
mx = MinMaxScaler()
df[['Area']] = mx.fit_transform(df[['Area']])

In [10]:
X = df.drop('Production', axis=1)
Y = df['Production']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=5)

In [11]:
rf = RandomForestRegressor().fit(X_train, y_train)
y_pred = rf.predict(X_test)
score = mean_squared_error(y_test, y_pred)
print('Error for rf '+str(score))

Error for rf 0.08810018335408522


In [18]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 400, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [None]
#max_depth.append()
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 49, 88, 127, 166, 205, 244, 283, 322, 361, 400], 'max_features': ['auto', 'sqrt'], 'max_depth': [None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [20]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 49, 88, 127, 166,
                                                         205, 244, 283, 322,
                                                         361, 400]},
                   random_state=42, verbose=2)

In [21]:
rf_random.best_params_

{'n_estimators': 361,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': None,
 'bootstrap': True}

In [24]:
y_pred1 = rf_random.predict(X_test)
score1 = mean_squared_error(y_test, y_pred1)
print('Error for rf '+str(score1))

Error for rf 0.08367494164706973


# KNeighborsRegressor

In [26]:
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)
y_pred2 = knn.predict(X_test)
score2 = mean_squared_error(y_test, y_pred2)
print('Error for rf '+str(score2))

Error for rf 0.10283755982760844


In [29]:
error = []
for i in range(1, 50):
    knn = KNeighborsRegressor(n_neighbors = i)
    knn.fit(X_train, y_train)
    y_pred3 = knn.predict(X_test)
    score3 = mean_squared_error(y_test, y_pred3)
    error.append(score3)
    

0.10090178709720582

In [36]:
print("Best n_neighbors value is", error.index(min(error)))

Best n_neighbors value is 3


3